# **EXPLORATORY DATA ANALYSIS OF FINANCIAL CONSUMER COMPLAINTS FOR BANK OF AMERICA(2017-2013)**

## DATA PRE-PROCESSING
Objective:
Clean and prepare the dataset to ensure it is accurate, consistent, and ready for meaningful analysis.

Analysis Approach:

- Handle missing or inconsistent entries

- Convert date fields into usable datetime formats

- Engineer new variables like year, month, and year–month

- Standardize product, issue, and response categories

- Prepare the final cleaned dataset for EDA

In [6]:
# Import pandas library

import pandas as pd
import numpy as np

print("Libraries imported successfully!")

Libraries imported successfully!


In [8]:
# Load the dataset

file_path = "/Users/arsha/Downloads/Consumer_Complaints.xlsx"
df = pd.read_excel(file_path)
print("Dataset Loaded Successfully!")

Dataset Loaded Successfully!


### Handle Missing Values
Identify and treat missing or null entries

In [21]:
print("Missing Values Summary:")
display(df.isnull().sum().sort_values(ascending=False))


Missing Values Summary:


Sub-issue                       10858
Company public response          2175
Timely response?                 1494
Sub-product                         7
Complaint ID                        0
Submitted via                       0
Date submitted                      0
Date received                       0
State                               0
Product                             0
Issue                               0
Company response to consumer        0
dtype: int64

Fill missing categorical values appropriately


In [25]:
df["Sub-product"].fillna("Unknown", inplace=True)
df["Timely response?"].fillna("Unknown", inplace=True)
df["Company public response"].fillna("No public response", inplace=True)

## *We keep 'Sub-issue' as NaN since not every complaint has one*

### Remove Duplicates
Each complaint should have a unique Complaint ID. Ensure no duplicate complaints are present.


In [26]:
duplicate_count = df.duplicated(subset="Complaint ID").sum()
print(f"Number of duplicate complaints: {duplicate_count}")

Number of duplicate complaints: 0


There is no duplicate values present in our dataset.


### Correct Data Types
Verify dates and categorical fields have appropriate formats.


In [36]:
# Convert date columns to datetime
df["Date received"] = pd.to_datetime(df["Date received"], errors="coerce")
df["Date submitted"] = pd.to_datetime(df["Date submitted"], errors="coerce")

In [29]:
# Confirm changes
print(df.dtypes)

Complaint ID                             int64
Submitted via                           object
Date submitted                  datetime64[ns]
Date received                   datetime64[ns]
State                                   object
Product                                 object
Sub-product                             object
Issue                                   object
Sub-issue                               object
Company public response                 object
Company response to consumer            object
Timely response?                        object
dtype: object


### Create Derived Columns
Add new variables for time-based and analytical insights.

In [30]:
# Extract temporal components for trend analysis
df["Year"] = df["Date received"].dt.year
df["Month"] = df["Date received"].dt.month_name()
df["Quarter"] = df["Date received"].dt.to_period("Q").astype(str)

In [31]:
# Calculate response time (days)
df["Response_Days"] = (df["Date submitted"] - df["Date received"]).dt.days

In [33]:
# Flag untimely responses
df["Untimely_Response_Flag"] = df["Timely response?"].apply(
    lambda x: 1 if str(x).strip().lower() == "no" else 0
)

In [34]:
# Simplify company response categories
def simplify_response(response):
    response = str(response).lower()
    if "monetary relief" in response:
        return "Monetary relief"
    elif "non-monetary relief" in response:
        return "Non-monetary relief"
    elif "closed with explanation" in response:
        return "Closed with explanation"
    elif "in progress" in response:
        return "In progress"
    else:
        return "Other"

df["Response_Category"] = df["Company response to consumer"].apply(simplify_response)

In [35]:
# Preview new derived columns
display(df[[
    "Date received", "Year", "Month", "Quarter", 
    "Response_Days", "Untimely_Response_Flag", "Response_Category"
]].head())

,Date received,Year,Month,Quarter,Response_Days,Untimely_Response_Flag,Response_Category
0,2021-10-27,2021,October,2021Q4,-3,0,Closed with explanation
1,2020-04-24,2020,April,2020Q2,0,0,Monetary relief
2,2022-07-27,2022,July,2022Q3,0,0,Closed with explanation
3,2023-07-11,2023,July,2023Q3,-1,0,In progress
4,2022-07-28,2022,July,2022Q3,-1,0,Closed with explanation


### Filtering & Aggregation
Prepare the data for analysis based on the project’s aim.

In [37]:
# Complaints by Year and Product
complaints_by_year_product = df.groupby(["Year", "Product"]).size().reset_index(name="Complaint_Count")


In [38]:
# Top 10 Complaint Issues
top_issues = df["Issue"].value_counts().head(10).reset_index()
top_issues.columns = ["Issue", "Complaint_Count"]

In [39]:
# Response Category Summary
response_summary = df["Response_Category"].value_counts().reset_index()
response_summary.columns = ["Response_Category", "Count"]

In [41]:
# Untimely Responses by Product
untimely_by_product = (
    df.groupby("Product")["Untimely_Response_Flag"].mean().reset_index()
    .rename(columns={"Untimely_Response_Flag": "Untimely_Response_Rate"})
    .sort_values(by="Untimely_Response_Rate", ascending=False)
)

### Display summaries

In [43]:
print("Complaints by Year and Product:")
display(complaints_by_year_product.head())

print("Top 10 Complaint Issues:")
display(top_issues)

print("Response Category Summary:")
display(response_summary)

print("Untimely Responses by Product:")
display(untimely_by_product)

Complaints by Year and Product:


,Year,Product,Complaint_Count
0,2017,Checking or savings account,2032
1,2017,Credit card or prepaid card,1123
2,2017,"Credit reporting, credit repair services, or o...",465
3,2017,Debt collection,285
4,2017,"Money transfer, virtual currency, or money ser...",141


Top 10 Complaint Issues:


,Issue,Complaint_Count
0,Managing an account,15109
1,Incorrect information on your report,4931
2,Problem with a purchase shown on your statement,4415
3,Closing an account,2953
4,Trouble during payment process,2827
5,Opening an account,2725
6,Problem with a lender or other company chargin...,2493
7,Fraud or scam,1987
8,Struggling to pay mortgage,1904
9,Problem with a credit reporting company's inve...,1876


Response Category Summary:


,Response_Category,Count
0,Closed with explanation,41044
1,Monetary relief,19970
2,In progress,1494
3,Other,8


Untimely Responses by Product:


,Product,Untimely_Response_Rate
3,Debt collection,0.063231
2,"Credit reporting, credit repair services, or o...",0.061608
8,Vehicle loan or lease,0.052133
1,Credit card or prepaid card,0.042539
4,"Money transfer, virtual currency, or money ser...",0.040544
0,Checking or savings account,0.034940
6,"Payday loan, title loan, or personal loan",0.030030
5,Mortgage,0.002424
7,Student loan,0.000000


In [45]:
#Summary
print("""
Data Pre-processing Complete!
- Missing values handled
- Duplicates checked
- Data types corrected
- Derived columns created (Year, Month, Quarter, Response_Days, etc.)
- Aggregated tables ready for EDA
""")


Data Pre-processing Complete!
- Missing values handled
- Duplicates checked
- Data types corrected
- Derived columns created (Year, Month, Quarter, Response_Days, etc.)
- Aggregated tables ready for EDA

